# Exploration

In [1]:
import pandas as pd
import numpy as np
folder_data = "/home/jeth/Projects/OMEA/POC/Poc_rfp_omea/cdr_data"
tower_df = pd.read_csv(f"{folder_data}/cell_towers_mali.csv")


### Voice 

In [ ]:
voice_df = pd.read_csv(f"{folder_data}/voice_cdr_mali_01.csv")
voice_df.head()

In [ ]:
# Silver transformations

# extract date (YYYY-MM-DD) from timestamp and insert as 'call_date' at column index 2
voice_df.insert(2, 'call_date', pd.to_datetime(voice_df['timestamp']).dt.date)

# extrat hour from timestamp and insert as 'call_hour' at column index 4
voice_df.insert(3, 'call_hour', pd.to_datetime(voice_df['timestamp']).dt.hour)

# compute call duration in minutes from seconds and insert at column index 7
voice_df.insert(7, 'duration_minutes', voice_df['duration_seconds'] / 60)

# derive a simple call status: NORMAL and USER_TERMINATED -> SUCCESS, otherwise FAILED
voice_df['call_status'] = np.where(
    voice_df['termination_reason'].isin(['NORMAL', 'USER_TERMINATED']),
    'SUCCESS',
    'FAILED'
)

# preview the first 10 rows to verify transformations
voice_df.head(10)

In [ ]:

# Gold aggregations
# compute traffic daily KPIs
voice_kpis = voice_df.groupby('call_date').apply(
    lambda g: pd.Series({
        'total_number_calls': g['call_id'].count(),
        'total_call_duration': g['duration_seconds'].sum(),
        'average_call_duration': g['duration_seconds'].mean(),
        'average_call_duration_success': g.loc[g['call_status'] == 'SUCCESS', 'duration_seconds'].mean(),
        'total_call_success': (g['call_status'] == 'SUCCESS').sum(),
        'total_call_failed': (g['call_status'] == 'FAILED').sum(),
        'total_revenue': g['charging_amount'].sum(),
        'total_duration_of_minutes': g['duration_minutes'].sum()
    }),
    include_groups=False
).reset_index()

voice_kpis.head()

In [ ]:
# compute traffic daily KPIs
voice_tower_kpis = voice_df.groupby(['call_date', 'call_hour', 'cell_id']).apply(
    lambda g: pd.Series({
        'total_number_calls': g['call_id'].count(),
        'total_call_duration': g['duration_seconds'].sum(),
        'total_call_success': (g['call_status'] == 'SUCCESS').sum(),
        'total_call_failed': (g['call_status'] == 'FAILED').sum(),
        'total_revenue': g['charging_amount'].sum()
    }),
    include_groups=False
).reset_index()

voice_tower_kpis.head()

---

### SMS

In [2]:
sms_df = pd.read_csv(f"{folder_data}/sms_cdr_mali_01.csv")

In [3]:
# extract date (YYYY-MM-DD) from timestamp and insert as 'call_date' at column index 2
sms_df.insert(2, 'sms_date', pd.to_datetime(sms_df['timestamp']).dt.date)
# extrat hour from timestamp and insert as 'call_hour' at column index 4
sms_df.insert(3, 'sms_hour', pd.to_datetime(sms_df['timestamp']).dt.hour)

sms_df.head()

,timestamp,sms_id,sms_date,sms_hour,sender_msisdn,receiver_msisdn,sms_type,message_length,cell_id,region,delivery_status,charging_amount
0,2024-12-14T08:00:00,SMS_000001,2024-12-14,8,22375359251,22373688162,MT,146,CELL_KAY_001,Kayes,DELIVERED,0.0
1,2024-12-14T08:00:16,SMS_000002,2024-12-14,8,22371347500,22370197394,MT,51,CELL_MOP_001,Mopti,DELIVERED,0.0
2,2024-12-14T08:00:27,SMS_000003,2024-12-14,8,22370536799,22373091816,MO,90,CELL_KAY_001,Kayes,DELIVERED,25.0
3,2024-12-14T08:00:35,SMS_000004,2024-12-14,8,22376299928,22379261127,MT,114,CELL_BAM_002,Bamako,DELIVERED,0.0
4,2024-12-14T08:00:39,SMS_000005,2024-12-14,8,22372058302,22370585904,MT,93,CELL_BAM_001,Bamako,DELIVERED,0.0


In [4]:

# Gold aggregations
# compute traffic daily KPIs
sms_kpis = sms_df.groupby('sms_date').apply(
    lambda g: pd.Series({
        'total_sms': g['sms_id'].count(),
        'total_delivered_sms': g.loc[g['delivery_status'] == 'DELIVERED', 'sms_id'].count(),
        'total_failed_sms': g.loc[g['delivery_status'] == 'FAILED', 'sms_id'].count(),
        'total_revenue': g['charging_amount'].sum(),
        'total_subscribers_sending_sms': g['sender_msisdn'].nunique(),
        'total_subscribers_receiving_sms': g['receiver_msisdn'].nunique()

    }),
    include_groups=False
).reset_index()

sms_kpis.head()

,sms_date,total_sms,total_delivered_sms,total_failed_sms,total_revenue,total_subscribers_sending_sms,total_subscribers_receiving_sms
0,2024-12-14,5473.0,4889.0,286.0,60350.0,5470.0,5472.0
1,2024-12-15,4527.0,4062.0,245.0,51175.0,4526.0,4526.0


In [5]:
sms_tower_kpis = sms_df.groupby(['sms_date', 'sms_hour', 'cell_id']).apply(
    lambda g: pd.Series({
        'total_sms': g['sms_id'].count(),
        'total_delivered_sms': g.loc[g['delivery_status'] == 'DELIVERED', 'sms_id'].count(),
        'total_failed_sms': g.loc[g['delivery_status'] == 'FAILED', 'sms_id'].count(),
        'total_revenue': g['charging_amount'].sum(),
        'total_subscribers_sending_sms': g['sender_msisdn'].nunique(),
        'total_subscribers_receiving_sms': g['receiver_msisdn'].nunique()

    }),
    include_groups=False
).reset_index()

sms_tower_kpis.head()

,sms_date,sms_hour,cell_id,total_sms,total_delivered_sms,total_failed_sms,total_revenue,total_subscribers_sending_sms,total_subscribers_receiving_sms
0,2024-12-14,8,CELL_BAM_001,31.0,25.0,4.0,375.0,31.0,31.0
1,2024-12-14,8,CELL_BAM_002,28.0,26.0,1.0,275.0,28.0,28.0
2,2024-12-14,8,CELL_BAM_003,28.0,24.0,2.0,225.0,28.0,28.0
3,2024-12-14,8,CELL_GAO_001,40.0,33.0,2.0,275.0,40.0,40.0
4,2024-12-14,8,CELL_KAY_001,32.0,30.0,1.0,500.0,32.0,32.0


### DATA

In [6]:
data_df = pd.read_csv(f"{folder_data}/data_cdr_mali_01.csv")

In [ ]:
data_df.insert(2, 'session_date', pd.to_datetime(data_df['timestamp']).dt.date)
# extrat hour from timestamp and insert as 'session_hour' at column index 4
data_df.insert(3, 'session_hour', pd.to_datetime(data_df['timestamp']).dt.hour)

data_df.head()

,timestamp,session_id,data_date,data_hour,msisdn,apn,session_duration_seconds,bytes_uploaded,bytes_downloaded,cell_id,region,session_end_reason,charging_amount
0,2024-12-14T08:00:00,DATA_000001,2024-12-14,8,22375229436,mms.mali,6906,5534048,10462930,CELL_SIK_001,Sikasso,NORMAL,1.6
1,2024-12-14T08:00:54,DATA_000002,2024-12-14,8,22372674723,mms.mali,1365,6920080,1482013,CELL_BAM_001,Bamako,USER_TERMINATED,0.0
2,2024-12-14T08:02:34,DATA_000003,2024-12-14,8,22376275890,mms.mali,3312,2113589,14276036,CELL_BAM_001,Bamako,NORMAL,1.6
3,2024-12-14T08:04:25,DATA_000004,2024-12-14,8,22376526392,wap.mali,4254,4644535,38985531,CELL_BAM_001,Bamako,NORMAL,6.4
4,2024-12-14T08:06:35,DATA_000005,2024-12-14,8,22370884000,mms.mali,3789,2444525,12327594,CELL_KID_001,Kidal,NORMAL,1.4


In [ ]:
data_kpis = data_df.groupby('session_date').apply(
    lambda g: pd.Series({
        'total_sessions': g['session_id'].count(),
        'total_active_sessions': g.loc[g['session_end_reason'] == 'NORMAL', 'session_id'].count(),
        'average_session_duration_sec': g['session_duration_seconds'].mean(),
        'average_session_duration_minutes': g['session_duration_seconds'].mean()/60,
        'average_session_duration_terminated_sec': g.loc[g['session_end_reason'] == 'USER_TERMINATED', 'session_duration_seconds'].mean(),
        'total_bytes_uploaded': g['bytes_uploaded'].sum(),
        'total_bytes_downloaded': g['bytes_downloaded'].sum(),
        'total_bytes': g['bytes_uploaded'].sum() + g['bytes_downloaded'].sum(),
        'total_data_volume_GB': (g['bytes_uploaded'].sum() + g['bytes_downloaded'].sum()) / (1024**3),  # in GB
        'average_throughput_per_session': ((g['bytes_uploaded'].sum() + g['bytes_downloaded'].sum())/g['session_id'].count()) / g['session_duration_seconds'],
        'total_revenue': g['charging_amount'].sum()

    }),
    include_groups=False
).reset_index()



data_kpis.head()

In [8]:
data_tower_kpis = data_df.groupby(['data_date', 'data_hour', 'cell_id']).apply(
    lambda g: pd.Series({
        'total_sessions': g['session_id'].count(),
        'total_active_sessions': g.loc[g['session_end_reason'] == 'NORMAL', 'session_id'].count(),
        'total_bytes_uploaded': g['bytes_uploaded'].sum(),
        'total_bytes_downloaded': g['bytes_downloaded'].sum(),
        'total_bytes': g['bytes_uploaded'].sum() + g['bytes_downloaded'].sum(),
        'total_data_volume_GB': (g['bytes_uploaded'].sum() + g['bytes_downloaded'].sum()) / (1024**3),  # in GB
        'total_revenue': g['charging_amount'].sum()

    }),
    include_groups=False
).reset_index()



data_tower_kpis.head()

,data_date,data_hour,cell_id,total_sessions,total_active_sessions,total_bytes_uploaded,total_bytes_downloaded,total_bytes,total_data_volume_GB,total_revenue
0,2024-12-14,8,CELL_BAM_001,8.0,7.0,105353851.0,1.081253e+09,1.186607e+09,1.105114,222.6
1,2024-12-14,8,CELL_BAM_002,4.0,3.0,128173036.0,1.656425e+09,1.784598e+09,1.662036,341.2
2,2024-12-14,8,CELL_BAM_003,3.0,3.0,57401540.0,4.277346e+08,4.851362e+08,0.451818,92.5
3,2024-12-14,8,CELL_GAO_001,4.0,3.0,70997428.0,9.571875e+08,1.028185e+09,0.957572,195.6
4,2024-12-14,8,CELL_KAY_001,5.0,3.0,36553687.0,1.627628e+08,1.993165e+08,0.185628,29.8


### NETWORK

In [9]:
technology_distribution = tower_df.groupby(['region', 'technology']).agg(
    count_towers=('cell_id', 'count')
).reset_index()

In [ ]:
technology_distribution.head()

## Draft

In [ ]:
# # Call volume and duration per day
# date_grouped = voice_df.groupby('call_date').agg(
#     total_number_calls = ('call_id', 'count'),
#     total_call_duration = ('duration_seconds', 'sum') where 'call_status' ==,
#     average_call_duration = ('duration_seconds', 'mean'),
#     total_call_success = ('call_status', lambda x: (x == 'SUCCESS').sum()),
#     total_call_failed = ('call_status', lambda x: (x == 'FAILED').sum()),
#     total_revenue = ('charging_amount', 'sum'),
#     total_duration_of_minutes = ('duration_minutes', 'sum')

# ).reset_index()
